In [1]:
import numpy as np
import pandas as pd
from itertools import combinations

In [2]:
def read_matrix(filename, sheet_name=0):
    """Read the matrix from an Excel (.xlsx) file."""
    df = pd.read_excel(filename, sheet_name=sheet_name)
    return df.values

In [3]:
def solve_system(rows_subset, matrix):
    """
    Given 5 row indices, create submatrix and solve the linear system.
    
    Steps:
    1. Extract 5 rows from matrix
    2. Apply logarithm to all entries
    3. Add a row of ones
    4. Solve Ax = b where A is first 6 columns, b is Index column with a one appended
    """
    # Extract the 5 rows
    submatrix = matrix[list(rows_subset), :]
    
    # Separate features (first 6 columns) and target (Index column)
    features = submatrix[:, :-1]  # Work, Money, Knowledge, Time, Power, Health
    target = submatrix[:, -1]      # Index
    
    # Apply logarithm to all entries
    features_log = np.log(features)
    target_log = np.log(target)
    
    # Add a row of ones (weights must sum to one)
    ones_row = np.ones((1, features_log.shape[1]))
    A = np.vstack([features_log, ones_row])
    
    # Extend target vector with a corresponding value
    # (appending 1)
    b = np.append(target_log, 1)
    
    # Solve the linear system Ax = b
    # Using least squares to gracefully handle singular cases (with tuple unpacking)
    x, residuals, rank, s = np.linalg.lstsq(A, b, rcond=None)
    
    return x

In [4]:
def main(filename, num_iterations='max'):
    """
    Main function to process the matrix and solve systems.
    
    Args:
        filename: Path to the input file
        num_iterations: Number of random 5-row selections to perform
    
    Returns:
        List of solution vectors
    """
    
    # Store all solutions
    solutions = []
    
    # Get number of rows and all possible combinations
    n_rows = matrix.shape[0]
    all_combinations = list(combinations(range(n_rows), 5))
    if num_iterations=='max':
        num_iterations = len(all_combinations)

    print(f"Matrix has {n_rows} rows")
    print(f"Total possible combinations: {len(all_combinations)}")
    print(f"Processing {num_iterations} iterations...")

    # Cycle through iterations
    for i in range(min(num_iterations, len(all_combinations))):
        # Pick 5 different rows
        if i < len(all_combinations):
            rows_subset = all_combinations[i]
        else:
            # If we've exhausted all combinations, sample randomly
            rows_subset = np.random.choice(n_rows, 5, replace=False)
        
        try:
            # Solve the system for this subset
            solution = solve_system(rows_subset, matrix)
            solutions.append(solution)
            
            if (i + 1) % 10000 == 0:
                print(f"Completed {i + 1} iterations")
        except Exception as e:
            print(f"Warning: Failed to solve system for rows {rows_subset}: {e}")
            continue
    
    print(f"\nTotal solutions found: {len(solutions)}")
    return solutions

In [5]:
# Path to the input file
input_file="GEI2025dataset.xlsx"
# Read the matrix
matrix = read_matrix(input_file)
# Number of random 5-row selections to perform
num_iterations='max'

In [6]:
# Run the analysis on all possible cases
solutions = main(input_file,num_iterations)

# Display some results
print("\nFirst 5 solutions:")
for i, sol in enumerate(solutions[:5]):
    formatted_sol = [f"{x:.15f}" for x in sol]
    print(f"Solution {i+1}: {formatted_sol}")

# Save solutions to a file
np.savetxt("solutions.txt", solutions, fmt='%.15f')
print("\nAll solutions saved to 'solutions.txt'")

Matrix has 28 rows
Total possible combinations: 98280
Processing 98280 iterations...
Completed 10000 iterations
Completed 20000 iterations
Completed 30000 iterations
Completed 40000 iterations
Completed 50000 iterations
Completed 60000 iterations
Completed 70000 iterations
Completed 80000 iterations
Completed 90000 iterations

Total solutions found: 98280

First 5 solutions:
Solution 1: ['0.158734577401797', '0.179224244291268', '0.130562504240674', '0.195712144807932', '0.180888266033566', '0.154878263224763']
Solution 2: ['0.143145001929047', '0.218861336160147', '0.126869571346099', '0.168646375795317', '0.189415847337597', '0.153061867431794']
Solution 3: ['0.154298660076618', '0.190502733743923', '0.129511703015436', '0.188010747557439', '0.183314736575660', '0.154361419030925']
Solution 4: ['0.172280706872679', '0.144782692034486', '0.133771375912953', '0.219230194540316', '0.173478460640208', '0.156456569999358']
Solution 5: ['0.162003305816077', '0.170913377284157', '0.13133681

In [7]:
def compute_geometric_means(solutions, matrix):
    """
    For each solution vector, apply it to all rows to compute geometric means.
    
    For each solution (x_0, ..., x_5) and each row (r_0, ..., r_5),
    compute: r_0^x_0 * r_1^x_1 * ... * r_5^x_5
    
    Args:
        solutions: List of solution vectors
        matrix: Original matrix with features in first 6 columns
    
    Returns:
        List of numpy arrays, where each array contains geometric means for all rows
    """
    results = []
    features = matrix[:, :-1]  # First 6 columns (Work, Money, Knowledge, Time, Power, Health)
    
    for solution in solutions:
        # For this solution, compute geometric mean for each row (with list comprehension)
        # Work in log space to avoid overflow
        # r_0^x_0 * ... * r_5^x_5 = exp(x_0*log(r_0) + ... + x_5*log(r_5))
        log_features = np.log(features)
        log_result = np.dot(log_features, solution)
        row_results = np.exp(log_result)
        results.append(row_results)
    
    return results

In [8]:
geometric_means = compute_geometric_means(solutions, matrix)

In [9]:
# Display some results
print("\nFirst 5 solutions:")
for i, sol in enumerate(solutions[:5]):
    print(f"Solution {i+1}: {sol}")


First 5 solutions:
Solution 1: [0.15873458 0.17922424 0.1305625  0.19571214 0.18088827 0.15487826]
Solution 2: [0.143145   0.21886134 0.12686957 0.16864638 0.18941585 0.15306187]
Solution 3: [0.15429866 0.19050273 0.1295117  0.18801075 0.18331474 0.15436142]
Solution 4: [0.17228071 0.14478269 0.13377138 0.21923019 0.17347846 0.15645657]
Solution 5: [0.16200331 0.17091338 0.13133682 0.20138713 0.17910025 0.15525911]


In [10]:
print("\nGeometric means for first 3 solutions (first 5 rows each):")
for i in range(min(3, len(geometric_means))):
    print(f"\nSolution {i+1} applied to all rows:")
    print(f"  First 5 rows: {geometric_means[i][:5]}")


Geometric means for first 3 solutions (first 5 rows each):

Solution 1 applied to all rows:
  First 5 rows: [63.4 68.5 58.1 53.2 71.8]

Solution 2 applied to all rows:
  First 5 rows: [63.4        68.5        58.1        53.2        71.88553266]

Solution 3 applied to all rows:
  First 5 rows: [63.4        68.5        58.1        53.2        71.82432742]


In [11]:
def find_closest_arrays(geometric_means, index_column, metric='euclidean'):
    """
    Find the numpy arrays closest to the Index column.
    
    Args:
        geometric_means: List of numpy arrays (output from compute_geometric_means)
        index_column: The Index column from the original matrix
        metric: Distance metric to use ('euclidean', 'mae', 'mse', 'rmse')
    
    Returns:
        Tuple of (best_index, best_array, best_distance, all_distances)
    """
    distances = []
    
    for arr in geometric_means:
        if metric == 'euclidean':
            # Euclidean distance
            dist = np.linalg.norm(arr - index_column)
        elif metric == 'mae':
            # Mean Absolute Error
            dist = np.mean(np.abs(arr - index_column))
        elif metric == 'mse':
            # Mean Squared Error
            dist = np.mean((arr - index_column) ** 2)
        elif metric == 'rmse':
            # Root Mean Squared Error
            dist = np.sqrt(np.mean((arr - index_column) ** 2))
        else:
            raise ValueError(f"Unknown metric: {metric}")
        
        distances.append(dist)
    
    # Find the index of the minimum distance
    best_idx = np.argmin(distances)
    best_array = geometric_means[best_idx]
    best_distance = distances[best_idx]
    
    return best_idx, best_array, best_distance, np.array(distances)

In [12]:
index_column = matrix[:, -1]
# Find the closest array to the Index column
best_idx, best_array, best_distance, all_distances = find_closest_arrays(
    geometric_means, index_column, metric='euclidean')

In [13]:
print("\n" + "="*60)
print("BEST FIT ANALYSIS")
print("="*60)
print(f"\nBest solution index: {best_idx}")
print(f"\nBest combination:")
n_rows = matrix.shape[0]
all_combinations = list(combinations(range(n_rows), 5))
print(all_combinations[best_idx])
print(f"Best distance (euclidean): {best_distance:.6f}")
print(f"\nBest solution coefficients:")
for j, coef in enumerate(solutions[best_idx]):
    print(f"  Coefficient {j}: {coef:.15f}")
print(f"\nComparison (first 10 rows):")
print(f"{'Index':<12} {'Predicted':<12} {'Error':<12}")
print("-" * 36)
for i in range(min(10, len(index_column))):
    error = best_array[i] - index_column[i]
    print(f"{index_column[i]:<12.4f} {best_array[i]:<12.4f} {error:<12.4f}")  


BEST FIT ANALYSIS

Best solution index: 36005

Best combination:
(2, 4, 14, 24, 27)
Best distance (euclidean): 0.176068

Best solution coefficients:
  Coefficient 0: 0.162511466842102
  Coefficient 1: 0.181002399159846
  Coefficient 2: 0.127268464919380
  Coefficient 3: 0.187055022839716
  Coefficient 4: 0.183763861249992
  Coefficient 5: 0.158398784988964

Comparison (first 10 rows):
Index        Predicted    Error       
------------------------------------
63.4000      63.4325      0.0325      
68.5000      68.4926      -0.0074     
58.1000      58.1000      0.0000      
53.2000      53.1685      -0.0315     
71.8000      71.8000      0.0000      
63.2000      63.2086      0.0086      
59.4000      59.4080      0.0080      
69.0000      69.0409      0.0409      
57.0000      57.0458      0.0458      
70.9000      70.8683      -0.0317     


In [14]:
print(f"\nComparison (all rows):")
print(f"{'Index':<12} {'Predicted':<12} {'Error':<12}")
print("-" * 36)
for i in range(len(index_column)):
    error = best_array[i] - index_column[i]
    print(f"{index_column[i]:<12.1f} {best_array[i]:<12.1f} {error:<12.2f}")



Comparison (all rows):
Index        Predicted    Error       
------------------------------------
63.4         63.4         0.03        
68.5         68.5         -0.01       
58.1         58.1         0.00        
53.2         53.2         -0.03       
71.8         71.8         0.00        
63.2         63.2         0.01        
59.4         59.4         0.01        
69.0         69.0         0.04        
57.0         57.0         0.05        
70.9         70.9         -0.03       
73.4         73.5         0.05        
57.1         57.0         -0.05       
61.9         61.9         -0.01       
47.6         47.5         -0.06       
56.7         56.7         0.00        
60.9         60.9         -0.04       
63.9         63.9         -0.02       
51.6         51.7         0.06        
58.9         58.9         -0.02       
69.5         69.5         0.03        
61.2         61.2         -0.04       
57.8         57.8         0.02        
63.4         63.4         0.03        
57.

In [15]:
df = pd.read_excel(input_file, sheet_name='dataframe')
df

,Country,Work,Money,Knowledge,Time,Power,Health,Index
0,EU-27,69.3,73.9,57.4,65.0,40.5,86.2,63.4
1,BE,69.1,78.6,56.3,76.3,49.9,86.3,68.5
2,BG,78.6,83.8,47.3,65.2,25.6,74.7,58.1
3,CZ,63.6,75.6,52.9,57.6,20.3,82.6,53.2
4,DK,70.0,82.3,55.3,81.1,57.3,87.5,71.8
5,DE,63.9,68.1,59.0,61.2,47.5,87.8,63.2
6,EE,76.5,82.9,50.7,74.4,21.9,86.8,59.4
7,IE,72.2,71.3,64.2,66.3,54.1,93.8,69.0
8,EL,67.9,76.7,58.7,56.0,26.2,83.8,57.0
9,ES,69.4,73.3,55.7,74.1,66.6,86.2,70.9


In [16]:
indices = all_combinations[best_idx]
df[df.index.isin(indices)]

,Country,Work,Money,Knowledge,Time,Power,Health,Index
2,BG,78.6,83.8,47.3,65.2,25.6,74.7,58.1
4,DK,70.0,82.3,55.3,81.1,57.3,87.5,71.8
14,LV,77.6,76.5,43.5,57.2,28.9,78.1,56.7
24,SI,73.3,86.2,45.0,63.4,24.6,86.6,58.0
27,SE,80.4,81.0,54.6,58.7,80.3,91.2,73.7
